In [1]:
import os
import logging
import sys
import struct
from array import array
import random
import numpy as np
from models.weight_transfer_model import *
from utils import classification_batch_evaluation, hist_loss_batch_eval, proto_episodic_performance, proto_performance


import tensorflow as tf
import os
keras = tf.keras

In [2]:
#######################1.)Boilerplate for loading Data#########################################################
############################################################################################################
class MNIST():
    def __init__(self, path):
        self.path = path

        self.test_img_fname = 't10k-images-idx3-ubyte'
        self.test_lbl_fname = 't10k-labels-idx1-ubyte'

        self.train_img_fname = 'train-images-idx3-ubyte'
        self.train_lbl_fname = 'train-labels-idx1-ubyte'

        self.test_images = []
        self.test_labels = []

        self.train_images = []
        self.train_labels = []

        self.num_classes = 10

    def load_testing(self):
        ims, labels = self.load(os.path.join(self.path, self.test_img_fname),
                                os.path.join(self.path, self.test_lbl_fname))

        self.test_images = self.process_images(ims)
        self.test_labels = self.process_labels(labels)

        return self.test_images, self.test_labels

    def load_training(self):
        ims, labels = self.load(os.path.join(self.path, self.train_img_fname),
                                os.path.join(self.path, self.train_lbl_fname))

        self.train_images = self.process_images(ims)
        self.train_labels = self.process_labels(labels)

        return self.train_images, self.train_labels

    def process_images(self, images):
        images_np = np.array(images) / 255.0
        return images_np

    def process_labels(self, labels):
        return np.array(labels)

    @classmethod
    def load(cls, path_img, path_lbl):
        with open(path_lbl, 'rb') as file:
            magic, size = struct.unpack(">II", file.read(8))
            if magic != 2049:
                raise ValueError('Magic number mismatch, expected 2049,'
                                 'got {}'.format(magic))

            labels = array("B", file.read())

        with open(path_img, 'rb') as file:
            magic, size, rows, cols = struct.unpack(">IIII", file.read(16))
            if magic != 2051:
                raise ValueError('Magic number mismatch, expected 2051,'
                                 'got {}'.format(magic))

            image_data = array("B", file.read())

        images = []
        for i in range(size):
            images.append([0] * rows * cols)

        for i in range(size):
            images[i][:] = image_data[i * rows * cols:(i + 1) * rows * cols]

        return images, labels

    def kntl_data_form(self, t1_train, t1_valid, k, n, t2_test):
        self.load_testing()
        self.load_training()

        self.x_train_lt5_full = self.train_images[self.train_labels < 5]
        self.y_train_lt5_full = self.train_labels[self.train_labels < 5]
        shuffle = np.random.permutation(len(self.y_train_lt5_full))
        self.x_train_lt5_full, self.y_train_lt5_full = self.x_train_lt5_full[shuffle], self.y_train_lt5_full[shuffle]

        x_test_lt5 = self.test_images[self.test_labels < 5]
        y_test_lt5 = self.test_labels[self.test_labels < 5]
        shuffle = np.random.permutation(len(y_test_lt5))
        self.x_test_lt5_full, self.y_test_lt5_full = x_test_lt5[shuffle], y_test_lt5[shuffle]

        self.x_lt5 = np.concatenate((self.x_train_lt5_full, self.x_test_lt5_full), axis=0)
        self.y_lt5 = np.concatenate((self.y_train_lt5_full, self.y_test_lt5_full), axis=0)

        print('Task 1 full: {0}'.format(len(self.x_lt5)))

        shuffle = np.random.permutation(len(self.y_lt5))
        self.x_lt5, self.y_lt5 = self.x_lt5[shuffle], self.y_lt5[shuffle]
        self.x_valid_lt5, self.y_valid_lt5 = self.x_lt5[:t1_valid], self.y_lt5[:t1_valid]
        self.x_train_lt5, self.y_train_lt5 = self.x_lt5[t1_valid:t1_valid + t1_train], self.y_lt5[t1_valid:t1_valid + t1_train]

        print('Task 1 training: {0}'.format(len(self.x_train_lt5)))
        print('Task 1 validation: {0}'.format(len(self.x_valid_lt5)))

        self.x_train_gte5_full = self.train_images[self.train_labels >= 5]
        self.y_train_gte5_full = self.train_labels[self.train_labels >= 5] - 5
        self.x_test_gte5_full = self.test_images[self.test_labels >= 5]
        self.y_test_gte5_full = self.test_labels[self.test_labels >= 5] - 5

        self.x_gte5 = np.concatenate((self.x_train_gte5_full, self.x_test_gte5_full), axis=0)
        self.y_gte5 = np.concatenate((self.y_train_gte5_full, self.y_test_gte5_full), axis=0)

        print('Task 2 full: {0}'.format(len(self.x_gte5)))

        self.x_train_gte5 = []
        self.y_train_gte5 = []
        classes = np.unique(self.y_gte5)
        chosen_classes = np.random.choice(classes, n, replace=False)
        for c in chosen_classes:
            idx = np.random.choice(np.where(self.y_gte5 == c)[0], k, replace=False)
            self.x_train_gte5.extend(self.x_gte5[idx])
            self.y_train_gte5.extend(self.y_gte5[idx])
            self.x_gte5 = np.delete(self.x_gte5, idx, axis=0)
            self.y_gte5 = np.delete(self.y_gte5, idx, axis=0)

        self.x_train_gte5 = np.array(self.x_train_gte5)
        self.y_train_gte5 = np.array(self.y_train_gte5)

        assert t2_test <= len(self.y_gte5)

        shuffle = np.random.permutation(len(self.y_gte5))
        self.x_gte5, self.y_gte5 = self.x_gte5[shuffle], self.y_gte5[shuffle]
        self.x_test_gte5, self.y_test_gte5 = self.x_gte5[:t2_test], self.y_gte5[:t2_test]

        print('k = {0}, n = {1}'.format(k, n))
        print('Task 2 training: {0}'.format(len(self.x_train_gte5)))
        print('Task 2 test: {0}\n'.format(len(self.x_test_gte5)))

        return (self.x_train_lt5, self.y_train_lt5), (self.x_valid_lt5, self.y_valid_lt5), (self.x_train_gte5, self.y_train_gte5), (self.x_test_gte5, self.y_test_gte5)

In [3]:
####################### 2.)Boilerplate for the WeighTtranfer model #########################################################
####################################################################################################################
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import functools
from functools import partial
import os
import sys
import tensorflow as tf
import pdb
import abc


ABC = abc.ABCMeta('ABC', (object,), {})

class Model(ABC):

    def __init__(self):
        super().__init__()

    def get_config(self, config):
        c = AttrDict()
        for k, v in config.items():
            c[k] = v
        return c

    @abc.abstractmethod
    def prediction(self):
        pass

    @abc.abstractmethod
    def optimize(self):
        pass
  
class AttrDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

def doublewrap(function):
    '''
    A decorator around a decorator allowing use of the original decorator
    without parentheses if no arguments are provided. All arguments
    must be optional.
    '''
    @functools.wraps(function)
    def decorator(*args, **kwargs):
        if len(args) == 1 and len(kwargs) == 0 and callable(args[0]):
            return function(args[0])
        else:
            return lambda wrapee: function(wrapee, *args, **kwargs)
    return decorator

@doublewrap
def define_scope(function, scope=None, *args, **kwargs):
    '''
    A decorator for functions that define Tensorflow operations. The wrapped
    function will only be executed once. Subsequent calls to it will directly
    return the result, so that operations are only added to the graph once.
    '''
    attribute = '_cache_' + function.__name__
    name = scope or function.__name__

    @property
    @functools.wraps(function)
    def decorator(self):
        if not hasattr(self, attribute):
            with tf.variable_scope(name, *args, **kwargs):
                setattr(self, attribute, function(self))
        return getattr(self, attribute)
    return decorator

def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

PS_OPS = [
    'Variable', 'VariableV2', 'AutoReloadVariable', 'MutableHashTable',
    'MutableHashTableOfTensors', 'MutableDenseHashTable'
]

def assign_to_device(device, ps_device):
    '''
    Returns a function to place variables on the ps_device.\
    If ps_device is not set, then the variables will be placed
    on the default device.
    '''
    def _assign(op):
        node_def = op if isinstance(op, tf.NodeDef) else op.node_def
        if node_def.op in PS_OPS:
            return ps_device
        else:
            return device
    return _assign

def _stride(s):
    return [1, s, s, 1]

def _relu(x):
    return tf.nn.relu(x)

def _sigmoid(x):
    return tf.nn.sigmoid(x)

def _tanh(x):
    return tf.nn.tanh(x)

def _softmax(x):
    return tf.nn.softmax(x)

def _max_pooling(name, x, kernel_size, stride, padding='SAME'):
    return tf.nn.max_pool(x, ksize=_stride(kernel_size), strides=_stride(stride), padding=padding, name=name)

def _conv(name, x, filter_size, in_size, out_size, stride, padding='SAME', bias=True, reuse=None, weight_decay=None):
    with tf.variable_scope(name, reuse=reuse):
        weights = tf.get_variable(
            'conv_weights', [filter_size, filter_size, in_size, out_size],
            tf.float32, initializer=tf.initializers.random_normal()
        )

        res = tf.nn.conv2d(x, weights, _stride(stride), padding=padding)

        if bias:
            biases = tf.get_variable(
                'conv_biases', [out_size], tf.float32,
                initializer=tf.initializers.zeros()
            )
            res += biases

        if weight_decay:
            wd = tf.nn.l2_loss(weights) * weight_decay
            tf.add_to_collection('weight_decay', wd)

    return res

def _fully_connected(name, x, out_size, reuse=None, weight_decay=None):
    with tf.variable_scope(name, reuse=reuse):
        weights = tf.get_variable(
            'fc_weights', [x.get_shape()[1], out_size], tf.float32,
            initializer=tf.initializers.random_normal()
        )
        biases = tf.get_variable(
            'fc_biases', [out_size], tf.float32, initializer=tf.initializers.zeros()
        )

        if weight_decay:
            wd = tf.nn.l2_loss(weights) * weight_decay
            tf.add_to_collection('weight_decay', wd)

        return tf.nn.xw_plus_b(x, weights, biases)


    

class WeightTransferModel(Model):

    def __init__(self, config,freeze):
        print("Inside the WeightTransferModel constructor function ",config)
        self.config = self.get_config(config)
        self.saver = None
        self.learning_rate = tf.placeholder(tf.float32)
        self.is_train = tf.placeholder(tf.bool)
        self.freeze =freeze

    def create_saver(self):
        self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

    def save_model(self, sess, step):
        # self.saver.save(sess, os.path.join(self.config.save_dir_by_rep, 'model.ckpt'), global_step=step)
        self.saver.save(sess, os.path.join(self.config.save_dir_by_rep, 'model.h5'), global_step=step)


    def restore_model(self, sess):
        checkpoint = tf.train.latest_checkpoint(self.config.save_dir_by_rep)
        if checkpoint is None:
            sys.exit('Cannot restore model that does not exist')
        self.saver.restore(sess, checkpoint)

    def get_single_device(self):
        devices = get_available_gpus()
        d = self.config.controller
        if devices:
            d = devices[0]
        return d

    @define_scope
    def optimize(self):
        ##Note that the scope of tf.GraphKeys is prediction
#         pass
        d = self.get_single_device()
        with tf.device(assign_to_device(d, self.config.controller)):
           
            pred = self.prediction
            cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, 
                labels=tf.one_hot(self.target, self.config.n)))
#             if self.freeze:
#                 #OLD APPROACH
#                 optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate,var_list=[self.x] )

    
#             else:
#                 optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
            optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
                  
         
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            
            
            ##Below command will freeze both the weight and biases for the convolution layers, they might or might not
            ## be used based on the below if condition :
            freeze_conv1  =tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"prediction/conv1")
            freeze_conv2  =tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES,"prediction/conv2")
            #pdb.set_trace()
            with tf.control_dependencies(update_ops):
                if self.freeze:
                    train_op = optimizer.minimize(cost,var_list=freeze_conv1+ freeze_conv2)
                else:
                    train_op = optimizer.minimize(cost)
    
            return train_op, cost

    @define_scope(scope='stream_metrics')
    def metrics(self):
        d = self.get_single_device()
        with tf.device(assign_to_device(d, self.config.controller)):
            pred = self.prediction
            acc, update_acc = tf.metrics.accuracy(self.target, tf.argmax(_softmax(pred), axis=1))
            return update_acc


class MNISTWeightTransferModel(WeightTransferModel):

    def __init__(self, config,freeze):
        super().__init__(config,freeze)
        self.input = tf.placeholder(tf.float32, [None, 784])
        self.target = tf.placeholder(tf.int32, [None])
        self.is_task1 = tf.placeholder(tf.bool)
        self.prediction
        self.optimize
        self.metrics
        self.freeze=freeze
        
        

    @define_scope
    def prediction(self):
        d = self.get_single_device()
        with tf.device(assign_to_device(d, self.config.controller)):
            x = self.input
            x = tf.reshape(x, [-1, 28, 28, 1])
            x = _relu(_conv('conv1', x, 3, x.get_shape()[-1], 32, 1))
            x = _max_pooling('pool2', _relu(_conv('conv2', x, 3, x.get_shape()[-1], 32, 1)), 2, 2)
            x = tf.contrib.layers.flatten(x)
            self.x = _relu(_fully_connected('fc1', x, 128))
            x1 = lambda: _fully_connected('fc3', x, self.config.n)
            x2 = lambda: _fully_connected('fc4', x, self.config.n)
            x = tf.cond(tf.equal(self.is_task1, tf.constant(True)), x1, x2)
            return x


In [4]:
####################### 3.)Other important methods#########################################################
####################################################################################################################

def train_classification(sess, model, data, params, weight_transfer=True):
    (x_train, y_train), (x_valid, y_valid), (x_train2, y_train2), (x_test2, y_test2) = data
    ##x_train = y_train = 8000 ;x_valid = y_valid=3000;x_train2 =y_train2 = 50 ; x_test2 =y_test2 =10000
    pdb.set_trace()
 
    temp_learning_rate_source_training = params['learning_rate']
    
    #pdb.set_trace()
    if weight_transfer:
        initial_best_epoch = {'epoch': -1, 'valid_acc': -1}
        

  
    
    with open("/home/abhishek/Desktop/{}_{}_{}.txt".format(params["dataset"],params["k"],params["n"]), "a") as f:
        for epoch in range(1, params['epochs'] + 1):
            shuffle = np.random.permutation(len(y_train))
            x_train, y_train = x_train[shuffle], y_train[shuffle]
            

            for i in range(0, len(y_train), params['batch_size']):
                x_train_mb, y_train_mb = x_train[i:i + params['batch_size']], y_train[i:i + params['batch_size']]
                
#                 sess.run(model.optimize, feed_dict={model.input: x_train_mb, model.target: y_train_mb, model.is_task1: True, model.is_train: True, model.learning_rate: params['learning_rate']})
                sess.run(model.optimize, feed_dict={model.input: x_train_mb, model.target: y_train_mb, model.is_task1: True, model.is_train: True, model.learning_rate: temp_learning_rate_source_training})
      
                

            valid_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], True, x_valid, y=y_valid, stream=True)

            print('valid [{} / {}] valid accuracy: {} learning Rate :{}'.format(epoch, params['epochs'] + 1, valid_acc,temp_learning_rate_source_training),file =f)
            logging.info('valid [{} / {}] valid accuracy: {}'.format(epoch, params['epochs'] + 1, valid_acc))

            if valid_acc > initial_best_epoch['valid_acc']:
                initial_best_epoch['epoch'] = epoch
                initial_best_epoch['valid_acc'] = valid_acc
                model.save_model(sess, epoch) 
                ##Saves the model at the following location : 
                ##trained_models/mnist/mnist_10_5/weight_transfer/replication1
                
                ##A
                ##Trying to save model as an HDF5 file
#                 model.save('my_model.h5')


            if epoch - initial_best_epoch['epoch'] >= params['patience']:
                print('Early Stopping Epoch: {}\n'.format(epoch))
                logging.info('Early Stopping Epoch: {}\n'.format(epoch))
                break

        print('Initial training done \n')
        logging.info('Initial training done \n')

        model.restore_model(sess) ##Restores the model after creating it .

    transfer_best_epoch = {'epoch': -1, 'train_acc': -1, 'test_acc': -1}
    es_acc = 0.0
#     model.freeze = True
#     print("Model parameters are ",model.freeze)


    with open("/home/abhishek/Desktop/{}_{}_{}.txt".format(params["dataset"],params["k"],params["n"]), "a") as f:
        temp_learning_rate_target_training = 0.01
        for epoch in range(1, params['epochs'] + 1):
            shuffle = np.random.permutation(len(y_train2))
            x_train2, y_train2 = x_train2[shuffle], y_train2[shuffle]
        
        
            if epoch%5==0:
                temp_learning_rate_target_training = temp_learning_rate_target_training *0.1
                
            for i in range(0, len(y_train2), params['batch_size']):
                x_train_mb, y_train_mb = x_train2[i:i + params['batch_size']], y_train2[i:i + params['batch_size']]
                sess.run(model.optimize, feed_dict={model.input: x_train_mb, model.target: y_train_mb, model.is_task1: False, model.is_train: True, model.learning_rate: params['learning_rate']})

            train_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], False, x_train2, y=y_train2, stream=True)

            print('train [{} / {}] train accuracy: {} learning Rate:{} '.format(epoch, params['epochs'] + 1, train_acc,temp_learning_rate_target_training),file=f)
            logging.info('train [{} / {}] train accuracy: {}'.format(epoch, params['epochs'] + 1, train_acc))

            if train_acc > transfer_best_epoch['train_acc']:
                transfer_best_epoch['epoch'] = epoch
                transfer_best_epoch['train_acc'] = train_acc
                test_acc = classification_batch_evaluation(sess, model, model.metrics, params['batch_size'], False, x_test2, y=y_test2, stream=True)
                transfer_best_epoch['test_acc'] = test_acc

            if epoch % params['patience'] == 0:
                acc_diff = transfer_best_epoch['train_acc'] - es_acc
                if acc_diff < params['percentage_es'] * es_acc:
                    print('Early Stopping Epoch: {}\n'.format(epoch))
                    logging.info('Early Stopping Epoch: {}\n'.format(epoch))
                    break
                es_acc = transfer_best_epoch['train_acc']

        print('Transfer training done \n')
        print('test accuracy: {}'.format(transfer_best_epoch['test_acc']))
        logging.info('Transfer training done \n')
        logging.info('test accuracy: {}'.format(transfer_best_epoch['test_acc']))
    
def get_model(params):
    model, data = None, None
    print("params:",params)
    if params['command'] == 'hist_loss':
        if params['dataset'] == 'mnist':
            model = MNISTHistModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletHistModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotHistModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetHistModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    elif params['command'] == 'proto':
        if params['dataset'] == 'mnist':
            model = MNISTProtoModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletProtoModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotProtoModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetProtoModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    elif params['command'] == 'weight_transfer':
        if params['dataset'] == 'mnist':
            model = MNISTWeightTransferModel(params,freeze = False)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'],params['k'], params['n'], params['t2_test'])
	    
	   
        elif params['dataset'] == 'isolet':
            model = IsoletWeightTransferModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotWeightTransferModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetWeightTransferModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
	    
    elif params['command'] == 'baseline':
        if params['dataset'] == 'mnist':
            model = MNISTBaselineModel(params)
            data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'], 
                params['k'], params['n'], params['t2_test'])
        elif params['dataset'] == 'isolet':
            model = IsoletBaselineModel(params)
            data = Isolet(params['data_path']).kntl_data_form(250, params['n'], params['k'], params['n'])
        elif params['dataset'] == 'omniglot':
            model = OmniglotBaselineModel(params)
            data = Omniglot(params['data_path']).kntl_data_form(params['n'], params['k'], params['n'])
        else:
            model = TinyImageNetBaselineModel(params)
            data = TinyImageNet(params['data_path']).kntl_data_form(350, params['n'], params['k'], params['n'])
    else:
        print('Unknown model type')
        logging.debug('Unknown model type')
        quit()
    #pdb.set_trace()
    return model, data

    

In [ ]:
####################### 4.)Running the code #########################################################
####################################################################################################################
params={'dataset': 'mnist', 'data_path': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/datasets/mnist', 't1_train': 8000, 't1_valid': 3000, 'k': 10, 'n': 5, 't2_test': 10000, 'epochs': 500, 'batch_size': 2048, 'learning_rate': 0.005, 'patience': 20, 'percentage_es': 0.01, 'random_seed': 1234, 'replications': 1, 'gpu': '0', 'controller': '/cpu:0', 'save_dir': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer', 'log_file': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer/log.txt', 'command': 'weight_transfer'}
data = MNIST(params['data_path']).kntl_data_form(params['t1_train'], params['t1_valid'],params['k'], params['n'], params['t2_test'])

random.seed(params['random_seed'])
initialization_seq = random.sample(range(50000), params['replications']) 
##initialization_seq = 28883

config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
for rep in range(params['replications']):
    tf.reset_default_graph()  #This does siemthing like clearing the default graph stack and i have no idea what does that mean 
    with tf.Session(config=config) as sess:
            #tf.set_random_seed(initialization_seq[rep])
        np.random.seed(initialization_seq[rep])
            
        model, data = get_model(params)
            #model : weight_transfer_model.MNISTWeightTransferModel
         
        assert not model is None
        assert not data is None
            
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
        params['init'] = init
        model.create_saver()
            #self.saver = tf.train.Saver(tf.global_variables(), max_to_keep=1)

        sess.run(init)
            
        rep_path = os.path.join(params['save_dir'], 'replication{}'.format(rep + 1))
        os.mkdir(rep_path)
        model.config['save_dir_by_rep'] = rep_path

        logging.debug('running training/testing')
        #pdb.set_trace()
        if params['command'] == 'baseline':
            train_classification(sess, model, data, params, weight_transfer=False)
        elif params['command'] == 'weight_transfer':
            train_classification(sess, model, data, params, weight_transfer=True)
        elif params['command'] == 'hist_loss':
            train_histogram_loss(sess, model, data, params)
        elif params['command'] == 'proto':
            train_proto_nets(sess, model, data, params)
        else:
            print('Unknown model type')
            logging.debug('Unknown model type')
            quit()
        #pdb.set_trace()
        print("Inside run(params): in run_model.py")


Task 1 full: 35735
Task 1 training: 8000
Task 1 validation: 3000
Task 2 full: 34265
k = 10, n = 5
Task 2 training: 50
Task 2 test: 10000

params: {'dataset': 'mnist', 'data_path': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/datasets/mnist', 't1_train': 8000, 't1_valid': 3000, 'k': 10, 'n': 5, 't2_test': 10000, 'epochs': 500, 'batch_size': 2048, 'learning_rate': 0.005, 'patience': 20, 'percentage_es': 0.01, 'random_seed': 1234, 'replications': 1, 'gpu': '0', 'controller': '/cpu:0', 'save_dir': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer', 'log_file': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/trained_models/mnist/mnist_10_5/weight_transfer/log.txt', 'command': 'weight_transfer'}
Inside the WeightTransferModel constructor function  {'dataset': 'mnist', 'data_path': '/home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/datasets/mnist', 't1_train': 8000, 't1_valid': 3000, 'k': 10,

W0915 03:11:11.712298 140655394404160 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0915 03:11:11.712993 140655394404160 deprecation.py:323] From /home/abhishek/anaconda3/lib/python3.7/site-packages/tensorflow/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.


Task 1 full: 35735
Task 1 training: 8000
Task 1 validation: 3000
Task 2 full: 34265
k = 10, n = 5
Task 2 training: 50
Task 2 test: 10000

> <ipython-input-4-9514fe38b625>(9)train_classification()
-> temp_learning_rate_source_training = params['learning_rate']
(Pdb) pdb.set_trace()
(Pdb) cont


W0915 03:11:36.379333 140655394404160 deprecation_wrapper.py:119] From /home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/utils.py:7: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0915 03:11:36.380294 140655394404160 deprecation_wrapper.py:119] From /home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/utils.py:7: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0915 03:11:36.381140 140655394404160 deprecation_wrapper.py:119] From /home/abhishek/Desktop/ANU/comp_6470/adapted_deep_embeddings/utils.py:8: The name tf.variables_initializer is deprecated. Please use tf.compat.v1.variables_initializer instead.

W0915 03:11:45.329012 140655394404160 deprecation.py:323] From /home/abhishek/anaconda3/lib/python3.7/site-packages/tensorflow/python/training/saver.py:960: remove_checkpoint (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instr


Program interrupted. (Use 'cont' to resume).
--Return--
> /home/abhishek/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py(1429)_call_tf_sessionrun()->[2.7551804]
-> run_metadata)


In [28]:
 with open("/home/abhishek/Desktop/a.txt") as f1:
                lines = f1.readlines()
                lines = [l for l in lines if "ROW" in l]
                with open("/home/abhishek/Desktop/mnist_10_5_q1_q4.txt", "w") as f:
                    f.writelines(lines)
#                     print("Targett Training Begins",file=f)

In [37]:
with open("/home/abhishek/Desktop/a.txt") as f:
    with open("/home/abhishek/Desktop/mnist_10_5_q1_q4.txt", "w") as f1:
        for x in f.readlines():
            f1.write(x)
            print("Hahah",file =f1)





In [34]:
with open("/home/abhishek/Desktop/mnist_10_5_q1_q4.txt", "w") as f:
        print("Hahah",file =f)

In [45]:
decay_after_epoch = 3
learning_rate=0.05
for epoch in range(1, 100):
    print((epoch-decay_after_epoch)%30)
    if epoch%decay_after_epoch==0 and epoch <=decay_after_epoch:
        learning_rate = learning_rate *0.1
        print("1st if")
    elif epoch-decay_after_epoch%30==0:
        print("elif")
        learning_rate = learning_rate *0.1

28
29
0
1st if
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
0
1
2
3
4
5
6


In [46]:
1.0000000000000006e-10 

1.0000000000000006e-10

In [47]:
1e-2

0.01